In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import math
import re
from datetime import datetime
from datetime import timedelta

In [2]:
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows',500)

In [3]:
play = pd.read_csv("play_by_play.csv")

In [4]:
play = play.drop(['eventnum', 'eventmsgactiontype',
       'wctimestring', 'neutraldescription', 'person1type',
       'player1_team_id', 'player1_team_city',
       'player1_team_nickname', 'player1_team_abbreviation', 'person2type',
       'player2_team_id', 'player2_team_city',
       'player2_team_nickname', 'player2_team_abbreviation', 'person3type',
       'player3_team_id', 'player3_team_city',
       'player3_team_nickname', 'player3_team_abbreviation',
       'video_available_flag'], axis = 1)
play

,game_id,eventmsgtype,period,pctimestring,homedescription,visitordescription,score,scoremargin,player1_id,player1_name,player2_id,player2_name,player3_id,player3_name
0,29600009,12,1,12:00,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN
1,29600009,10,1,12:00,Jump Ball Olajuwon vs. Polynice: Tip to Williamson,NaN,NaN,NaN,165,Hakeem Olajuwon,178,Olden Polynice,722,Corliss Williamson
2,29600009,5,1,11:49,NaN,Williamson Traveling Turnover (P1.T1),NaN,NaN,722,Corliss Williamson,0,NaN,0,NaN
3,29600009,2,1,11:27,MISS Olajuwon 13' Jump Shot,NaN,NaN,NaN,165,Hakeem Olajuwon,0,NaN,0,NaN
4,29600009,4,1,11:23,Drexler REBOUND (Off:1 Def:0),NaN,NaN,NaN,17,Clyde Drexler,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227006,22201014,8,4,0:01,SUB: Porter Jr. FOR Smith,NaN,NaN,NaN,202397,Ish Smith,1629008,Michael Porter Jr.,0,NaN
13227007,22201014,8,4,0:01,NaN,SUB: Claxton FOR Johnson,NaN,NaN,1629661,Cameron Johnson,1629651,Nic Claxton,0,NaN
13227008,22201014,2,4,0:00,MISS Jokic 25' 3PT Turnaround Jump Shot,NaN,NaN,NaN,203999,Nikola Jokic,0,NaN,0,NaN
13227009,22201014,4,4,0:00,NUGGETS Rebound,NaN,NaN,NaN,1610612743,NaN,0,NaN,0,NaN


In [5]:
def calc_time(start_time, end_time):
    time = re.compile("\d+")
    start_time = time.findall(start_time)
    end_time = time.findall(end_time)
    delta = timedelta(seconds = float(start_time[1]) - float(end_time[1]), minutes = float(start_time[0]) - float(end_time[0]))
    return math.ceil(delta.total_seconds()/60)

In [6]:
player = pd.read_csv("all_data.csv")
player["rebounds"] = [np.nan] * player.shape[0]
player["ORB"] = [0] * player.shape[0]
player["DRB"] = [0] * player.shape[0]
player["MP"] = [0] * player.shape[0]
player["start_time"] = [np.nan] * player.shape[0]
player

,Unnamed: 0,person_id,name,field_goals_made,field_goals_ast,field_goals_missed,times_ball_lost,times_ball_stolen,fouls,games_played,field_goals_blocked,free_throws_made,free_throws_missed,rebounds,points,three-pointers,ORB,DRB,MP,start_time
0,0,722.0,Corliss Williamson,3354.0,917.0,3479.0,1200.0,430.0,2110.0,745.0,242.0,1813.0,713.0,NaN,6505.0,101.0,0,0,0,NaN
1,1,165.0,Hakeem Olajuwon,2008.0,649.0,2046.0,769.0,454.0,933.0,361.0,629.0,922.0,373.0,NaN,5170.0,59.0,0,0,0,NaN
2,2,17.0,Clyde Drexler,837.0,719.0,1078.0,340.0,242.0,350.0,141.0,77.0,472.0,133.0,NaN,5368.0,76.0,0,0,0,NaN
3,3,53.0,Mario Elie,923.0,917.0,1103.0,476.0,343.0,687.0,347.0,50.0,619.0,104.0,NaN,25307.0,271.0,0,0,0,NaN
4,4,51.0,Mahmoud Abdul-Rauf,616.0,311.0,784.0,154.0,79.0,249.0,155.0,8.0,149.0,27.0,NaN,14071.0,196.0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,1631123.0,Jamaree Bouyea,6.0,4.0,8.0,4.0,4.0,6.0,5.0,2.0,1.0,1.0,NaN,7591.0,45.0,0,0,0,NaN
2496,2496,1631245.0,Quenton Jackson,2.0,1.0,1.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,NaN,134.0,2.0,0,0,0,NaN
2497,2497,1630649.0,Stanley Umude,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,NaN,8110.0,59.0,0,0,0,NaN
2498,2498,1630548.0,Johnny Juzang,6.0,1.0,16.0,1.0,1.0,6.0,5.0,3.0,0.0,2.0,NaN,81.0,0.0,0,0,0,NaN


In [ ]:
count = 0
game_id = 0
period = 1
game_id_index = 0
eventmsgtype = 1
period_index = 2
timestring = 3
homedescription = 4
visitordescription = 5
player1_id = 8
player1_name = 9
player2_id = 10
player2_name = 11
player3_id = 12
player3_name = 13
pattern = re.compile("\d+")
print("starting...")
for row in play.values:
    data = {'person_id' : 0, 'points' : 0}
    if (pd.notna(row[player1_name])) & True in (player.loc[(player["name"] == row[player1_name]), "start_time"]).isna().values:
        player.loc[(player["name"] == row[player1_name]), "start_time"] = row[timestring]
            
    if (pd.notna(row[player2_name])) & True in (player.loc[(player["name"] == row[player2_name]), "start_time"]).isna().values:
        player.loc[(player["name"] == row[player2_name]), "start_time"] = row[timestring]  
                
    if (pd.notna(row[player3_name])) & True in (player.loc[(player["name"] == row[player3_name]), "start_time"]).isna().values:
        player.loc[(player["name"] == row[player3_name]), "start_time"] = row[timestring]
            
    if row[eventmsgtype] == 4:
        if pd.notna(row[homedescription]):
            player.loc[(player["person_id"] == row[player1_id]), "rebounds"] = row[homedescription]
        else:
            player.loc[(player["person_id"] == row[player1_id]), "rebounds"] = row[visitordescription]

    elif row[eventmsgtype] == 8:
        try:
            MP = calc_time(player.loc[(player["person_id"] == row[player1_id]), "start_time"].values[0], row[timestring])
            player.loc[(player["person_id"] == row[player1_id]), "MP"] += MP
            player.loc[(player["person_id"] == row[player1_id]), "start_time"] = np.nan
        except:
            pass
    
    elif row[eventmsgtype] == 13:
        try:
            for i in player["start_time"].values:
                if type(i) == str:
                    MP = calc_time(i, row[timestring])
                    player.loc[player["start_time"] != np.nan, "MP"] += MP
                    player.loc[player["start_time"] != np.nan, "start_time"] = np.nan
            period = row[period_index]
        except:
            pass
        
    if count % 100000 == 0:
        print(count, end = ", ")
    count += 1
    
    if game_id != row[game_id_index]:
        try:
            for values in player.loc[player["rebounds"].notna()].values:
                temp = pattern.findall(values[13])
                player.loc[(player["name"] == values[2]), "ORB"] += int(temp[0])
                player.loc[(player["name"] == values[2]), "DRB"] += int(temp[1])
            game_id = row[game_id_index]
        except:
            pass
    
print("Finished")

starting...
0, 100000, 200000, 

In [ ]:
player.head(50)

In [ ]:
from pathlib import Path  
filepath = Path('Lumiere_Reasearch/all_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
all_in_one.to_csv(filepath)  